# 🐳 Develop a PCM for the Gulf Stream Extension region

Notebook toward a [Profile Classification Models (PCM)](https://pyxpcm.readthedocs.io/en/latest/overview.html) for the Gulf Stream.

# ✅ Step 1: assemble a training set

Using [argopy](https://argopy.readthedocs.io/), we load all QC=1 T/S data from the Gulf Stream Extension region, from 2000/01/01 to 2021/12/31.

Since this represents more than 39000 profiles, we run the entire pre-processing on monthly chunk of data (and saved netcdf files for each months).

Then we load the monthly timeseries of final profiles and assemble a single dataset with all data in 1 file saved in ``/docs/data/Argo-GulfStreamBox-prof-sdl.nc``.

⚠  Profiles are interpolated on standard 10db pressure levels from 0 to 1500db. Hence all profiles not reaching 1500db are dropped out.


⚠  You should use a deepest level that will be deep enough to handle the vertical axis of the PCM in step 2. I.e. if you choose 1500db in here, you won't be able to train a PCM on a 0-2000db vertical axis. And on the other hand, if you want a 0-500db PCM, then using 1500db here will make you miss a lot of profiles ... This is a compromise between the speed of assembling a training set and trials in PCM development phase.
    

In [1]:
import sys, os
import numpy as np
import xarray as xr
xr.set_options(display_style="html", display_expand_attrs=False);
import pandas as pd

sys.path.insert(0, "/Users/gmaze/git/github/euroargodev/argopy")
import argopy
argopy.clear_cache()
argopy.status(refresh=5)
argopy.set_options(ftp='/Volumes/CORIOLIS-GDAC')  # Use to get the index and estimate to nb of profiles in the area

sys.path.insert(0, "/Users/gmaze/git/github/euroargodev/boundary_currents_pcm")
# sys.path.insert(0, os.sep.join([os.getcwd(), '..', '..']))  # From binder
from pcmbc.utilities import from_misc_pres_2_std_depth, process_list_in_parallel
from pcmbc.plot import Plotter

/Users/gmaze/miniconda3/envs/virtualfleet/lib/python3.8/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


HTML(value='<table><tr><td><img src="https://img.shields.io/static/v1?style=flat-square&label=src%20argovis%20…

# 🔹  Define where to save and name of output files

In [2]:
data_output_folder = 'data'  # Relative to this notebook file location
domain_nickname = 'GulfStream'  # This will be used to name all files created in this notebook

# domain_nickname = 'WestMed'  # This will be used to name all files created in this notebook

# 🔹  Define the domain to download data for

The box here should encompasses a larger area that the core of BC current, in order to show to the classifier enough profiles with different structures.

In [3]:
# [lon_min, lon_max, lat_min, lat_max]

box = [-75.,-35.,35,50.]  # Gulf Stream Extension

# box = [-1., 5., 35., 42.]  # Western Mediterranean (Extended)
# box = [-0.5, 5.6, 38.8, 41.4]  # Western Mediterranean
# box = [-10., -6., 32., 38.5]  # Gulf of Cadiz (Extended)
# box = [-20., 5. ,69., 77.]  # Nordic Seas (western part)
# box = [-20., 15. , 65., 77.]  # Nordic Seas (Extended)

Indicate below the maximum pressure ALL profiles will be required to reach to be selected:

⚠  You should use a deepest level that will be deep enough to handle the vertical axis of the PCM in step 2. I.e. if you choose 1500db in here, you won't be able to train a PCM on a 0-2000db vertical axis. And on the other hand, if you want a 0-500db PCM, then using 1500db here will make you miss a lot of profiles ... This is a compromise between the speed of assembling a training set and trials in PCM development phase.

In [4]:
max_pressure = 1500

# 🔹 Load Argo data with argopy

In [5]:
# Before downloading the data, make sure it will be of reasonable size for memory !

index_box = box.copy()
index_box.append('2000-01-01')
index_box.append('2022-01-01')

index = argopy.IndexFetcher(src='gdac', cache=True).region(index_box).load()
print("%i max profiles reported for your data selection" % index.fetcher.N_FILES)
if index.fetcher.N_FILES > 5e3:
    print("""\n OH OH !\nYou have a significant amount of profiles to load, depending 
          on your data-source this may take a while or fail, or succeed ! Be warned !""")

39210 max profiles reported for your data selection

 OH OH !
You have a significant amount of profiles to load, depending 
          on your data-source this may take a while or fail, or succeed ! Be warned !


# 🔹 Deal with a large amount of profiles

## load/process/save data for each month

In [6]:
if not os.path.exists(data_output_folder):
    os.mkdir(data_output_folder)
else:
    print("✅ : output folder already exists ('%s')" % data_output_folder)
    
data_output_folder_montly = os.sep.join([data_output_folder, 'monthly'])
if not os.path.exists(data_output_folder_montly):
    os.mkdir(data_output_folder_montly)
else:
    print("✅ : output folder already exists ('%s')" % data_output_folder_montly)

✅ : output folder already exists ('data')
✅ : output folder already exists ('data/monthly')


In [7]:
def process_one_monthly_chunk(item=None):
    """

    Given the global variable ``box``, ``max_pressure``, ``data_output_folder``, ``domain_nickname``, 
    this function will:
    
    - load data with argopy (using src=argovis)
    - transform points to profiles
    - interp data on standard pressure levels
    
    Data are saved after each steps.
    
    Parameters
    ----------
    item: :class:`pandas:Timestamp`        
        Timestamp with the last day of the month to process. eg: 2000-01-31
        
    Returns
    -------
    The final number of profiles saved or None
    """    
    data_box = box.copy()
    data_box.append(0)   # Pres_min
    data_box.append(max_pressure)# Pres_max
    data_box.append(item.strftime('%Y-%m-01'))
    data_box.append((item + pd.Timedelta("24 hours")).strftime('%Y-%m-%d'))

    # Load points data:
    data_file = os.sep.join([data_output_folder, 'monthly', 'Argo-%s-%s.nc' % (domain_nickname, item.strftime('%Y%m'))])
    should_continue = True
    if not os.path.exists(data_file):
        # print("🔜 %s" % data_file)
        try:
            argo = argopy.DataFetcher(src='argovis', cache=True, parallel=True).region(data_box).load()
            ds = argo.data
            ds.to_netcdf(data_file)
            # print("✅ done !")
        except Exception:
            should_continue = False
            pass
    else:
        # print("👌 %s already exists" % data_file)
        ds = xr.open_dataset(data_file)
        
    # Transform points to profiles:
    data_file = os.sep.join([data_output_folder, 'monthly', 'Argo-%s-%s-prof.nc' % (domain_nickname, item.strftime('%Y%m'))])
    if not os.path.exists(data_file) and should_continue:
        # print("🔜 %s" % data_file)
        try:
            dsp = ds.argo.point2profile()
            dsp.to_netcdf(data_file)
            # print("✅ done !")
        except Exception:
            should_continue = False
            pass
    elif should_continue:
        # print("👌 %s already exists" % data_file)
        dsp = xr.open_dataset(data_file)
        
    # Interp on standard pressure levels:
    standard_pressure_levels = np.arange(0, data_box[5], 10)
    data_file = os.sep.join([data_output_folder, 'monthly', 'Argo-%s-%s-prof-sdl.nc' % (domain_nickname, item.strftime('%Y%m'))])
    if not os.path.exists(data_file) and should_continue:
        # print("🔜 %s" % data_file)
        try:
            dsi = dsp.argo.interp_std_levels(std_lev=standard_pressure_levels, axis='PRES')
            for v in dsi:
                dsi[v].attrs = dsp[v].attrs
            dsi.to_netcdf(data_file)
            # print("✅ done !")
        except Exception:
            should_continue = False
            pass
    elif should_continue:
        # print("👌 %s already exists" % data_file)
        dsi = xr.open_dataset(data_file)
        
    return len(dsi['N_PROF']) if should_continue else None

In [8]:
# month_list = pd.date_range(start='2000-01-01', end='2022-01-01', freq='M')
month_list = pd.date_range(start=index_box[4], end=index_box[5], freq='M')

# process_one_monthly_chunk(month_list[-1])
data, month_failed = process_list_in_parallel(process_one_monthly_chunk, month_list, errors='raise')
print("%i profiles interpolated correctly !" % np.sum(np.array([d for d in data if d])))

  3%|███▋                                                                                                                                         | 7/264 [00:55<10:55,  2.55s/it]/Users/gmaze/git/github/euroargodev/argopy/argopy/xarray.py:1094: UserWarning: None of the profiles can be interpolated (not reaching the requested depth range).
  warnings.warn(
/Users/gmaze/git/github/euroargodev/argopy/argopy/xarray.py:1094: UserWarning: None of the profiles can be interpolated (not reaching the requested depth range).
  warnings.warn(
  5%|██████▎                                                                                                                                     | 12/264 [01:01<08:18,  1.98s/it]/Users/gmaze/git/github/euroargodev/argopy/argopy/xarray.py:1094: UserWarning: None of the profiles can be interpolated (not reaching the requested depth range).
  warnings.warn(
  9%|█████████████▎                                                                                        

3066 profiles interpolated correctly !


## open and concat all monthly data files into a single file

In [9]:
# Get the list of data files to open and concatenate:
import glob
file_list = sorted(glob.glob(os.sep.join([data_output_folder, 'monthly', 'Argo-%s-2*-prof-sdl.nc' % domain_nickname])))
print("%i files to load and merge" % len(file_list))
print("> total size on disk: %0.2f Megabytes" % (np.sum(np.array([os.path.getsize(f) for f in file_list]))*1e-6))

88 files to load and merge
> total size on disk: 13.77 Megabytes


In [10]:
# Load all data (monthly files)
data, failed = process_list_in_parallel(lambda item: xr.open_dataset(item), file_list, errors='raise')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:01<00:00, 46.22it/s]


In [11]:
# Merge them all in a single dataset:
dsi = xr.concat(data,
       dim='N_PROF',
       data_vars='minimal',
       coords='minimal',
       compat='override')
dsi = dsi.assign_coords({'N_PROF': np.arange(len(dsi['N_PROF']))})
for v in dsi:
    dsi[v].attrs = data[0][v].attrs  # Preserve attributes
dsi

<xarray.Dataset>
Dimensions:            (N_PROF: 3066, PRES_INTERPOLATED: 150)
Coordinates:
  * N_PROF             (N_PROF) int64 0 1 2 3 4 5 ... 3061 3062 3063 3064 3065
    TIME               (N_PROF) datetime64[ns] 2003-07-01T08:02:00 ... 2020-1...
    LATITUDE           (N_PROF) float64 43.63 37.95 42.77 ... 37.46 48.93 35.64
    LONGITUDE          (N_PROF) float64 -58.34 -65.79 -58.81 ... -36.1 -53.22
  * PRES_INTERPOLATED  (PRES_INTERPOLATED) int64 0 10 20 30 ... 1470 1480 1490
Data variables:
    CYCLE_NUMBER       (N_PROF) int32 37 61 25 26 38 26 ... 154 114 102 12 164
    DATA_MODE          (N_PROF) object 'D' 'D' 'D' 'D' 'D' ... 'R' 'D' 'A' 'D'
    DIRECTION          (N_PROF) object 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER    (N_PROF) int32 4900136 4900133 ... 6901604 5904772
    PRES               (N_PROF, PRES_INTERPOLATED) float64 9.0 10.0 ... 1.49e+03
    PSAL               (N_PROF, PRES_INTERPOLATED) float64 32.08 32.06 ... 35.0
    TEMP               (N_PROF, PRES_INTERPOLATED) float64 12.24 11.82 ... 4.714
Attributes: (8)

In [12]:
# Save full dataset on file:
dsi.to_netcdf(os.sep.join([data_output_folder,'Argo-%s-prof-sdl.nc' % domain_nickname]))

In [13]:
dsi.to_zarr(os.sep.join([data_output_folder, 'Argo-%s-prof-sdl.zarr' % domain_nickname]))

***
This notebook has been developed within the framework of the Euro-ArgoRISE project. This project has received funding from the European Union’s Horizon 2020 research and innovation programme under grant agreement no 824131. Call INFRADEV-03-2018-2019: Individual support to ESFRI and other world-class research infrastructures.

<a href="https://www.euro-argo.eu/EU-Projects/Euro-Argo-RISE-2019-2022">
<img src="https://user-images.githubusercontent.com/59824937/146353317-56b3e70e-aed9-40e0-9212-3393d2e0ddd9.png" width="75"/>
</a>
